In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Solnechnyy,50.72,136.63,31.73,95,34,3.40,RU,2020-11-01 23:23:26
1,1,Ushuaia,-54.80,-68.30,59.00,54,75,18.34,AR,2020-11-01 23:19:43
2,2,Mar Del Plata,-38.00,-57.56,61.00,67,0,8.05,AR,2020-11-01 23:21:30
3,3,Sao Felix Do Xingu,-6.64,-51.99,76.32,82,92,0.31,BR,2020-11-01 23:23:26
4,4,Bengkulu,-3.80,102.27,75.45,88,99,4.56,ID,2020-11-01 23:21:14


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Sao Felix Do Xingu,-6.64,-51.99,76.32,82,92,0.31,BR,2020-11-01 23:23:26
4,4,Bengkulu,-3.80,102.27,75.45,88,99,4.56,ID,2020-11-01 23:21:14
9,9,Cayenne,4.93,-52.33,78.80,100,97,3.36,GF,2020-11-01 23:23:27
12,12,Olinda,-8.01,-34.86,80.60,74,40,8.05,BR,2020-11-01 23:23:27
14,14,Kavieng,-2.57,150.80,85.69,67,1,5.75,PG,2020-11-01 23:21:55
24,24,Alyangula,-13.85,136.42,87.80,45,7,5.82,AU,2020-11-01 23:23:28
25,25,Georgetown,5.41,100.34,78.80,88,20,3.36,MY,2020-11-01 23:22:38
26,26,Hithadhoo,-0.60,73.08,83.34,74,100,9.42,MV,2020-11-01 23:20:38
28,28,Atuona,-9.80,-139.03,77.47,79,68,15.12,PF,2020-11-01 23:23:28
32,32,Sao Filipe,14.90,-24.50,79.21,76,75,11.48,CV,2020-11-01 23:22:25


In [10]:
preferred_cities_df.count()

City_ID       159
City          159
Lat           159
Lng           159
Max Temp      159
Humidity      159
Cloudiness    159
Wind Speed    159
Country       159
Date          159
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Sao Felix Do Xingu,BR,76.32,-6.64,-51.99,
4,Bengkulu,ID,75.45,-3.80,102.27,
9,Cayenne,GF,78.80,4.93,-52.33,
12,Olinda,BR,80.60,-8.01,-34.86,
14,Kavieng,PG,85.69,-2.57,150.80,
24,Alyangula,AU,87.80,-13.85,136.42,
25,Georgetown,MY,78.80,5.41,100.34,
26,Hithadhoo,MV,83.34,-0.60,73.08,
28,Atuona,PF,77.47,-9.80,-139.03,
32,Sao Filipe,CV,79.21,14.90,-24.50,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Sao Felix Do Xingu,BR,76.32,-6.64,-51.99,Apart Hotel Rio Xingu
4,Bengkulu,ID,75.45,-3.80,102.27,Grage Hotel Bengkulu
9,Cayenne,GF,78.80,4.93,-52.33,Hôtel Le Dronmi
12,Olinda,BR,80.60,-8.01,-34.86,Hotel 5 Sóis
14,Kavieng,PG,85.69,-2.57,150.80,Nusa Island Retreat
24,Alyangula,AU,87.80,-13.85,136.42,Groote Eylandt Lodge
25,Georgetown,MY,78.80,5.41,100.34,Cititel Penang
26,Hithadhoo,MV,83.34,-0.60,73.08,Scoop Guest House
28,Atuona,PF,77.47,-9.80,-139.03,Villa Enata
32,Sao Filipe,CV,79.21,14.90,-24.50,Tortuga B&B


In [14]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Add Info Box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))